# 1. Browser-use: 자율 탐색 도구 마스터하기

이번 실습에서 메인으로 다룰 무기는 **Browser-use**라는 오픈소스 프레임워크입니다. (공식 GitHub: [browser-use](https://github.com/browser-use/browser-use))

Browser-use는 에이전트가 사람처럼 웹 브라우저를 직접 제어할 수 있게 해주는 파이썬 라이브러리입니다. 단순히 데이터를 긁어오는 '크롤링'을 넘어, AI가 웹사이트의 버튼을 클릭하고, 정보를 검색하고, 복잡한 워크플로우를 스스로 수행하게 만드는 웹 에이전트(Web Agent) 개발의 핵심 도구입니다.

### 주요 기능

- 지능적 브라우징: 정해진 스크립트가 아니라, AI가 화면을 보고 다음 행동(클릭, 입력, 스크롤 등)을 스스로 판단합니다.
- Playwright 기반: 강력한 브라우저 자동화 도구인 Playwright를 사용하여 Chrome, Firefox 등 다양한 브라우저를 제어합니다.
- 간편한 통합: LangChain과 같은 프레임워크와 잘 연동되며, Google, OpenAI, Anthropic의 최신 모델을 비전(Vision) 기능을 통해 활용합니다.
- 멀티 탭 지원: 여러 탭을 동시에 관리하며 복잡한 정보를 비교하거나 수집할 수 있습니다.

### 작동 원리

- 관찰(Observe): 현재 웹 페이지의 HTML 구조와 스크린샷을 찍어 AI에게 전달합니다.
- 생각(Think): AI가 사용자의 목표(예: "가장 저렴한 항공권 예약해줘")와 현재 페이지 상태를 분석합니다.
- 행동(Act): 분석 결과를 바탕으로 클릭, 타이핑 등 구체적인 브라우저 액션을 실행합니다.


### CUA(Computer-Use Agent)의 일종

우리가 쓸 `Browser-use`가 바로 이 CUA 개념을 웹 브라우저 환경에 구현한 겁니다. *"다나와 가서 RTX 4090 최저가 모델 3개만 찾아와"* 라고 자연어로 던지면, 알아서 다나와 켜고, 검색하고, 클릭해서 결과를 가져옵니다.

.env 파일을 로드하여 API를 환경변수에 등록합니다.

In [1]:
import asyncio
import os, sys
from dotenv import load_dotenv
from browser_use import Agent, Browser, ChatGoogle

import nest_asyncio

nest_asyncio.apply()

# .env 파일 로드
load_dotenv("/mnt/d/agent_research/hands_on/.env", override=True)

# 프로젝트 경로 추가
sys.path.append("/mnt/d/agent_research/hands_on")

# VNC용 디스플레이 설정 (start_vnc.sh에서 설정한 값과 일치해야 함)
os.environ["DISPLAY"] = ":1"

/home/hukim/env_langchain_123/lib/python3.12/site-packages/requests/__init__.py:113: RequestsDependencyWarning: urllib3 (2.6.3) or chardet (6.0.0.post1)/charset_normalizer (3.4.4) doesn't match a supported version!
  warnings.warn(


### 첫 번째 자율 탐색: 네이버 뉴스 요약하기

자, 이제 컴퓨터에게 첫 번째 임무를 줘보겠습니다. 
아래 코드는 아무런 HTML 태그 분석 없이, 단순히 **"네이버 뉴스에 들어가서 기사 하나를 요약해 줘"**라는 자연어 명령만을 내립니다. 
`Agent`는 다음과 같은 과정을 스스로 거칩니다:

1. 브라우저를 열고 네이버 뉴스에 접속 (Action)
2. 뉴스 목록을 확인 (Observe)
3. 흥미로운 기사를 클릭 (Act)
4. 기사 본문을 읽고 요약 (Think & Done)
Jupyter Notebook 환경에서는 비동기 충돌이 일어날 수 있으므로 `nest_asyncio`를 적용했습니다. 코드를 실행하고 브라우저가 어떻게 스스로 움직이는지, 그리고 로그(Log) 창에서 AI가 어떤 '생각(Thought)'을 거쳐 행동하는지 관찰해 보세요!

In [2]:
from browser_use import Agent, ChatGoogle
from dotenv import load_dotenv
import asyncio


async def basic_query():
    llm = ChatGoogle(model="gemini-flash-latest")
    task = "네이버 뉴스에 접속해서 첫 번째 기사를 들어가서 관련 내용을 소개해줘"
    agent = Agent(task=task, llm=llm)
    result = await agent.run()
    return result

if __name__ == "__main__":
    result = asyncio.run(basic_query())

INFO     [service] Using anonymized telemetry, see https://docs.browser-use.com/development/monitoring/telemetry.
INFO     [Agent] 🎯 Task: 네이버 뉴스에 접속해서 첫 번째 기사를 들어가서 관련 내용을 소개해줘
INFO     [Agent] Starting a browser-use agent with version 0.11.12, with provider=google and model=gemini-flash-latest
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: Initial step, no previous goal to evaluate.
INFO     [Agent]   🧠 Memory: Starting the task to navigate to Naver News and summarize the first article.
INFO     [Agent]   🎯 Next goal: Navigate to Naver News.
INFO     [Agent]   ▶️   navigate: url: https://news.naver.com, new_tab: False
WARNING  [BrowserSession] ⚠️ Page readiness timeout (4.0s, 4071ms) for https://news.naver.com
INFO     [tools] 🔗 Navigated to https://news.naver.com
INFO     [Agent] 📋 Plan updated with 4 steps
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Successfully navigated to Naver News. Verdict: Success
INFO     [Agent]   🧠 Mem

In [ ]:
# history: 여러 개의 AgentHistory 객체들이 담긴 리스트
history_list = result.history 

# 총 몇 스텝(Step)을 거쳤는지 확인
print(f"총 {len(result.history)} 스텝을 진행했습니다.")


In [ ]:
# 마지막 스텝 정보 가져오기
last_step = result.history[-1]

print("1) LLM이 내놓은 모델의 상태 (생각/추론 과정)")
last_model_output = last_step.model_output
if last_model_output:
    print("당시 기억 : ", last_model_output.current_state.memory)  # 에이전트의 당시 기억
    print("다음 목표 : ", last_model_output.current_state.next_goal) # 에이전트의 다음 목표

print("2) LLM이 선택한 액션(명령어)")
if last_model_output and last_model_output.action:
    # 어떤 액션을 했는지 딕셔너리 형태로 변환해서 보기
    print([action.model_dump(exclude_unset=True) for action in last_model_output.action])

print("3) 에이전트가 위치했던 URL")
print(last_step.state.url) # 에이전트가 위치했던 URL

print("4) 액션 수행 후 결과")
for res in last_step.result:
    print(res.extracted_content) # 화면에서 추출한 글자 내용 
    print(res.success) # 작업 성공 여부


최종 결과만 추출

browser-use 에이전트는 실행 결과를 깔끔하게 정리해주는 메서드를 제공합니다.

In [ ]:
# 최종 추출된 결과 텍스트만 쏙 빼오기
print(result.final_result())

# 작업이 성공적으로 잘 끝났는지 (True / False / None)
print(result.is_successful())

# 작업 중 발생한 에러 메시지들만 리스트로 모아오기
print(result.errors())

# 방문했던 모든 URL들 순서대로 확인하기
print(result.urls())


### 비용 관리: 분석 비용 트래킹하기

In [ ]:
from utils.cost_tracker import CostTracker

gemini-flash-lastest 모델 사용

In [ ]:
async def cost_track_browser_use1():

    tracker = CostTracker()

    llm = ChatGoogle(model="gemini-flash-latest")
    task = "네이버 뉴스에 접속해서 하나의 기사를 들어가서 관련 내용을 소개해줘"
    
    # 💡 핵심 1: calculate_cost=True 옵션을 켭니다.
    agent = Agent(
        task=task, 
        llm=llm,
        calculate_cost=True
    )
    
    # Agent 실행 후 결과(history) 반환받기
    history = await agent.run()
    
    # ==========================================
    # 💡 핵심 2: 비용 및 토큰 사용량(UsageSummary) 출력
    # ==========================================
    if history.usage:
        print("\n📊 [토큰 및 비용 사용량 요약]")
        print(f"- 전체 사용 토큰: {history.usage.total_tokens} tokens")
        print(f"  └ 입력(Prompt) 토큰: {history.usage.total_prompt_tokens} tokens")
        print(f"  └ 출력(Completion) 토큰: {history.usage.total_completion_tokens} tokens")
        print(f"- 총 발생 비용: ${history.usage.total_cost:.6f}") # 소수점 자리수 확장

        # 트래커를 통해 파일에 누적 기록하기
        tracker.record_usage(task, history.usage)
    else:
        print("\n비용 계산에 실패했거나 측정되지 않았습니다.")
if __name__ == "__main__":
    asyncio.run(cost_track_browser_use1())

gemini-flash-lastest 모델 사용

In [ ]:
async def cost_track_browser_use2():

    tracker = CostTracker()

    llm = ChatGoogle(model="gemini-2.0-flash")
    task = "네이버 뉴스에 접속해서 하나의 기사를 들어가서 관련 내용을 소개해줘"
    
    # 💡 핵심 1: calculate_cost=True 옵션을 켭니다.
    agent = Agent(
        task=task, 
        llm=llm,
        calculate_cost=True
    )
    
    # Agent 실행 후 결과(history) 반환받기
    history = await agent.run()
    
    # ==========================================
    # 💡 핵심 2: 비용 및 토큰 사용량(UsageSummary) 출력
    # ==========================================
    if history.usage:
        print("\n📊 [토큰 및 비용 사용량 요약]")
        print(f"- 전체 사용 토큰: {history.usage.total_tokens} tokens")
        print(f"  └ 입력(Prompt) 토큰: {history.usage.total_prompt_tokens} tokens")
        print(f"  └ 출력(Completion) 토큰: {history.usage.total_completion_tokens} tokens")
        print(f"- 총 발생 비용: ${history.usage.total_cost:.6f}") # 소수점 자리수 확장

        # 트래커를 통해 파일에 누적 기록하기
        tracker.record_usage(task, history.usage)
    else:
        print("\n비용 계산에 실패했거나 측정되지 않았습니다.")
if __name__ == "__main__":
    asyncio.run(cost_track_browser_use2())

### 심화 기능 활용하기

##### (1) 기존 브라우저 연결

지금까지는 AI가 실행될 때마다 아무 기록이 없는 '새 브라우저(샌드박스)'를 띄웠습니다. 하지만 실무에서는 이미 로그인이 다 되어 있는 내 컴퓨터의 브라우저를 그대로 조종해야 할 때가 많습니다. 

이 코드는 **현재 여러분 컴퓨터에 설치된 실제 구글 크롬**을 찾아 연결합니다. (미리 크롬 브라우저를 모두 닫아두고 실행해 주세요.)

In [ ]:
def get_default_chrome_profile() -> str | None:
    profiles = Browser.list_chrome_profiles()
    
    if not profiles:
        print("❌ 설치된 크롬 프로필을 찾을 수 없습니다.")
        return None
        
    print(f"✅ 발견된 크롬 프로필: {profiles[0]['name']}")
    return profiles[0]['directory']

In [ ]:
get_default_chrome_profile()

In [ ]:
async def real_browser_query():
    # 1. 내 시스템 기반의 Browser 객체 생성
    profile_path = get_default_chrome_profile()
    
    # 2. 에이전트 실행 시 생성한 브라우저 객체를 주입
    browser = Browser.from_system_chrome(profile_directory=profile_path)
    llm = ChatGoogle(model="gemini-2.0-flash")
    
    task = "아마존(amazon.com)에 들어가서 화이트보드 마커펜을 검색해 줘."
    
    agent = Agent(
        task=task, 
        llm=llm, 
        browser=browser
    )
    
    result = await agent.run()
    
    # 세션 종료 후 브라우저 닫기
    await browser.stop()
    return result

result = asyncio.run(real_browser_query())
print(result.final_result())

##### (2) 로그인 세션 정보 저장

매번 내 브라우저 전체를 연결하는 것은 무거울 수 있습니다. 이 코드는 로그인 정보(쿠키 등)만 파일로 저장합니다.

In [ ]:
import asyncio
from browser_use import Browser

async def export_auth():
    # 1. 내 크롬 브라우저를 띄워 로그인 상태를 확인합니다.
    browser = Browser.from_system_chrome()
    
    await browser.start()
    
    # 2. 현재 로그인 세션(쿠키, 로컬 스토리지 등)을 JSON 파일로 저장합니다.
    await browser.export_storage_state('storage_state.json')
    
    await browser.stop()
    print('✅ 로그인 세션이 storage_state.json에 저장되었습니다.')

if __name__ == '__main__':
    asyncio.run(export_auth())

##### (3) 개발자 도구 사용

AI가 복잡한 폼 입력을 실수할 때, 개발자가 직접 작성한 Playwright 코드를 명령어로 내리는 방식입니다.

In [ ]:
from pydantic import BaseModel, Field
from browser_use import Agent, ChatGoogle, Tools
from browser_use.agent.views import ActionResult

# 1. AI가 사용할 커스텀 도구 정의
class PreciseClickParams(BaseModel):
    selector: str = Field(..., description="클릭할 요소의 CSS 셀렉터")

tools = Tools()

@tools.registry.action("정밀 클릭 도구: 일반적인 클릭이 안 될 때 셀렉터를 직접 지정해 클릭합니다.", param_model=PreciseClickParams)
async def precise_click(params: PreciseClickParams, browser_session):
    # browser_session을 통해 직접 Playwright 페이지 객체에 접근 가능합니다.
    page = await browser_session.get_current_page()
    await page.click(params.selector)
    return ActionResult(extracted_content=f"{params.selector}를 성공적으로 클릭했습니다.")

async def run_custom_agent():
    agent = Agent(
        task="네이버 로그인을 시도하고, 안 되면 'precise_click' 도구를 사용해줘",
        llm=ChatGoogle(model='gemini-flash-latest'),
        tools=tools # 내가 만든 도구 전달
    )
    await agent.run()

if __name__ == '__main__':
    asyncio.run(run_custom_agent())

##### (4) 데이터 form 전송

이번에는 정보를 폼에 채워서 전송해봅시다.

In [ ]:
import asyncio
from browser_use import Agent, ChatGoogle

async def run_form_filling():
    llm = ChatGoogle(model='gemini-flash-latest')
    
    # 작업 정의: 사이트 방문 -> 데이터 입력 -> 제출 -> 결과 확인
    task = """
    https://httpbin.org/forms/post 에 접속해서 다음 정보로 폼을 채워줘:
    - 이름: 홍길동
    - 전화번호: 010-1234-5678
    - 이메일: test@example.com
    - 사이즈: Medium / 토핑: cheese / 시간: 지금
    - 의견: Browser-use 테스트 중입니다.
    
    폼을 제출한 후 서버로부터 받은 응답 내용을 요약해서 알려줘.
    """
    
    agent = Agent(task=task, llm=llm)
    await agent.run()

if __name__ == '__main__':
    asyncio.run(run_form_filling())

##### (5) 페이지 내 데이터 추출

기본적인 데이터 추출을 명령해봅니다.

In [ ]:
async def run_data_extraction():
    llm = ChatGoogle(model='gemini-flash-latest')
    
    # 5개의 명언, 저자, 태그를 특정 형식으로 추출 요청
    task = """
    https://quotes.toscrape.com/ 에서 다음 정보를 추출해줘:
    - 페이지에 있는 첫 5개의 명언
    - 각 명언의 저자와 태그들
    
    결과는 '명언 1: [내용] - 저자: [이름] - 태그: [태그들]' 형식으로 정리해줘.
    """
    
    agent = Agent(task=task, llm=llm)
    await agent.run()

if __name__ == '__main__':
    asyncio.run(run_data_extraction())

##### (6) 에이전트의 빠른 브라우징 작업

에이전트의 불필요한 생각 시간을 줄이고 브라우저 동작 속도를 극대화하는 설정입니다.

- flash_mode=True: AI의 내부 사고 과정을 생략하여 응답 속도를 높입니다.
- BrowserProfile: 페이지 로딩 대기 시간과 액션 간격을 0.1초로 단축합니다.
- extend_system_message: AI에게 "최대한 짧고 빠르게 행동하라"는 지침을 추가합니다.

In [ ]:
from browser_use import BrowserProfile

async def run_fast_agent():
    llm = ChatGoogle(model='gemini-flash-latest')
    
    # 1. 속도 최적화 브라우저 설정
    fast_profile = BrowserProfile(
        minimum_wait_page_load_time=0.1, # 최소 대기 시간 단축
        wait_between_actions=0.1,        # 동작 간격 단축
        headless=False
    )
    
    # 2. 모델에게 내리는 '빨리빨리' 명령
    optimization_prompt = "최대한 간결하게 응답하고, 가능한 여러 동작을 한 번에 실행해서 목표에 빠르게 도달해."

    agent = Agent(
        task="네이버 뉴스 상위 3개 기사 제목만 빠르게 뽑아줘.",
        llm=llm,
        flash_mode=True,                # 사고 과정 생략
        browser_profile=fast_profile,   # 최적화 프로필 적용
        extend_system_message=optimization_prompt
    )
    
    await agent.run()

if __name__ == '__main__':
    asyncio.run(run_fast_agent())

##### (7) 비디오 녹화

작업 과정을 녹화합니다. AI가 어떻게 움직였는지 사후에 검토하거나 디버깅할 때 매우 유용합니다.

In [ ]:
import asyncio
from pathlib import Path
from browser_use import Agent, Browser, ChatGoogle
from browser_use.browser import BrowserProfile
from dotenv import load_dotenv

load_dotenv()

async def main():
    # 1. 영상이 저장될 폴더 설정
    video_dir = Path('./recordings')
    video_dir.mkdir(exist_ok=True)
    
    # 2. 브라우저 설정 (핵심: record_video_dir)
    # 구조를 따름
    browser = Browser(
        record_video_dir=video_dir  # 영상 저장 경로 지정
    )

    # 3. 브라우저 프로필 설정 (창을 띄워서 확인하기 위해 headless=False)
    browser_profile = BrowserProfile(
        headless=False
    )

    # 4. LLM 및 에이전트 설정
    llm = ChatGoogle(model='gemini-flash-latest')
    
    task = "https://www.google.com 에 접속해서 'browser-use'라고 검색한 뒤 첫 번째 결과를 클릭해줘."

    agent = Agent(
        task=task,
        llm=llm,
        browser_session=browser,
        browser_profile=browser_profile
    )

    print(f"🚀 작업을 시작합니다. 녹화본은 '{video_dir}' 폴더에 저장됩니다.")

    try:
        # 에이전트 실행 (단계 제한을 둬서 테스트)
        await agent.run(max_steps=10)
    finally:
        print(f"✅ 작업 완료! '{video_dir}' 폴더에서 mp4 파일을 확인하세요.")

if __name__ == '__main__':
    asyncio.run(main())